In [1]:
import os
import sys
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src import *

In [2]:
process_directory_real_data(os.getcwd())

In [10]:


def process_results(base_path):
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(base_path, 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index
    leadership_log_likelihood_df = pd.read_csv(os.path.join(base_path, 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index

    return log_likelihood_df, leadership_log_likelihood_df

In [12]:
unique_datasets = pd.read_csv(os.path.join(os.path.join(os.getcwd(), 'results'), 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['dataset_id'].isin(unique_datasets)]
print(filtered_dataset_info)



    dataset_id                               name     N         M  K1   K2
5            8               glasgow_city_council   209    188376   1   13
6            9               AGH_course_selection    10       299   7    9
9           12                            t_shirt    12        30  11   11
12          18               minneapolis_election   778    137482   1    3
13          24               mechanical_turk_dots    13      3183   4    4
14          25             mechanical_turk_puzzle    15      3180   4    4
15          28                       APA_election     6    203896   1    5
16          30                UK_labor_party_vote     6       266   1    5
18          34                      cities_survey    85       784   6    6
20          38               project_bidding_data   148       309   5    6
22          42                             boxing   466      5594   1   17
25          45                     tennis_ranking   701      2911  55  100
32          52           

In [14]:

log_like, leadership_log = process_results(os.path.join(os.getcwd(), 'results'))


In [15]:
print(log_like)

<bound method DataFrame.reset_index of              newman  newman_leadership  higher_order_newman  \
dataset                                                       
8         -3.149264          -3.253266            -3.145642   
9         -9.090856         -11.111998            -9.033880   
12       -15.622417         -22.406003           -15.501611   
18        -1.426642          -1.429877            -1.426507   
24        -2.980528          -3.000419            -2.980088   
25        -2.963186          -2.985208            -2.962320   
28        -3.688126          -3.697533            -3.687508   
30        -2.520300          -2.561292            -2.516799   
34        -5.286676          -6.034788            -5.279423   
38        -5.509632          -6.234310            -5.425675   
42       -21.598977         -37.398478           -20.808323   
45      -265.238368        -382.647541          -258.656654   
52       -46.151124         -69.369585           -45.658969   
53       -33.840

In [16]:
print(leadership_log)

<bound method DataFrame.reset_index of            newman  newman_leadership  higher_order_newman  \
dataset                                                     
8       -1.145382          -1.116115            -1.146650   
9       -0.796780          -2.328810            -0.875689   
12      -2.167541          -2.026527            -2.213226   
18      -0.926906          -0.925245            -0.927667   
24      -1.253703          -1.249427            -1.257059   
25      -1.231324          -1.225800            -1.234210   
28      -1.390431          -1.388304            -1.391609   
30      -1.175482          -1.181198            -1.173605   
34      -1.218744          -1.224927            -1.235766   
38      -1.777837          -1.914692            -1.764380   
42      -1.368041          -0.786595            -1.535461   
45      -2.401018          -1.533752            -2.630939   
52      -2.478443          -2.098105            -2.602823   
53      -2.444009          -2.334592          